<a href="https://colab.research.google.com/github/maskot1977/tmd2022/blob/JjYuuNRj8Fef/%E7%AC%AC2%E5%9B%9E%EF%BC%9A%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E9%81%A9%E7%94%A8%E7%AF%84%E5%9B%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

「AI創薬・ケモインフォマティクス入門」講義資料　（講師：小寺正明）

1月28日（土）19:40～21:10 第2回「モデルの適用範囲」

# 第2回：モデルの適用範囲

## 各種設定

### Google Drive に学習履歴を保存するための設定

In [ ]:
# 以下の２つの設定は必要に応じて適宜調整してください。
dateflag = "0117"  # 解析日を記録するための変数
MODEL_PATH = "./drive/MyDrive/tmd2022-2/"  # データの保存場所を指定するための変数
learning_time_limit = 600  # １つの学習器あたりに許す最大の学習時間（秒）
timeout_optuna = 600  # Optuna による反復計算に許す最大の学習時間（秒）
n_trials_optuna = 2  # Optuna による反復計算の最大回数（普通は100や1000などの数字を入れる）

In [ ]:
# Google Colaboratory から Google Drive にマウント

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# もしデータ保存場所がなければ作る

import os

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

### Optuna のインストール

In [ ]:
!pip install optuna

## １次元データ



In [ ]:
# 二峰性の乱数
import numpy as np

X = np.sort(
    np.concatenate(
        [np.random.normal(25, 10, (10, 1)), np.random.normal(75, 10, (20, 1))]
    ).flatten()
).reshape(-1, 1)

In [ ]:
X

In [ ]:
import matplotlib.pyplot as plt

plt.hist(X)
plt.show()

In [ ]:
plt.scatter(X, [-0.5 for x in X], alpha=0.5)
plt.hist(X)
plt.show()

## K近傍法（k-NearestNeighbors, k-NN）

In [ ]:
from sklearn.neighbors import NearestNeighbors

n_neighbors = 3  # 自分自身を含めて k 番目に近い点までを「近傍」とみなす
model = NearestNeighbors(n_neighbors=n_neighbors)  # モデルの立ち上げ
model.fit(X)  # X のデータ構造を学習

In [ ]:
distances, indices = model.kneighbors(X)

In [ ]:
indices  # 自分自身を含む「近隣」インデックスを、自分自身から近い順に

In [ ]:
distances  # 「近隣」までの距離

In [ ]:
distances[:, n_neighbors - 1]  # n_neighbors 番目の近隣までの距離

In [ ]:
sorted(distances[:, n_neighbors - 1])  # n_neighbors 番目の近隣までの距離の小さい順にソート

In [ ]:
out = 0.1
int((len(X) - 1) * (1 - out))  # out の割合を除外したい時の区切りは何番目か

In [ ]:
out = 0.1
threshold = sorted(distances[:, n_neighbors - 1])[int((len(X) - 1) * (1 - out))]
threshold  # out の割合を除外したい時の閾値はいくらか

In [ ]:
# 閾値より下が「適用範囲」
x_latent = np.linspace(0, 100, 101).reshape(-1, 1)
plt.plot(
    x_latent,
    model.kneighbors(x_latent)[0][:, n_neighbors - 1],
    label="k neighbor distance",
)
plt.plot([0, 100], [threshold, threshold], label="threshold")
plt.scatter(X, [-0.5 for x in X], alpha=0.5, label="data")
plt.legend()
plt.show()

### One Class SVM (OCSVM)

In [ ]:
from sklearn.svm import OneClassSVM

model = OneClassSVM()  # モデルの立ち上げ
model.fit(X)  # X のデータ構造を学習

In [ ]:
# 大きいほど、データ密度が高い
score = model.decision_function(X)
score

In [ ]:
out = 0.2
threshold = sorted(score)[int((len(X) - 1) * out)]
threshold

In [ ]:
# 閾値より上が「適用範囲」
x_latent = np.linspace(0, 100, 101).reshape(-1, 1)
plt.plot(x_latent, model.decision_function(x_latent))
plt.plot([0, 100], [threshold, threshold])
plt.scatter(X, [1 for x in X], alpha=0.5)
plt.show()

## ２次元データ

In [ ]:
# ４種類の２次元サンプリングデータを用意します
import matplotlib.pyplot as plt
import numpy as np

X1 = []  # dense サンプリングデータ
for x in np.linspace(-7, 7, 19):
    for y in np.linspace(-7, 7, 19):
        X1.append([x, y])
X1 = np.array(X1)

X2 = []  # sparse サンプリングデータ
for x in np.linspace(-7, 7, 7):
    for y in np.linspace(-7, 7, 7):
        X2.append([x, y])
X2 = np.array(X2)

X3 = []  # O-shape サンプリングデータ
for x in np.linspace(-7, 7, 18):
    for y in np.linspace(-7, 7, 18):
        r = np.sqrt(x ** 2 + y ** 2)
        if 4 < r and r < 6:
            X3.append([x, y])
X3 = np.array(X3)

X4 = []  # X-shape サンプリングデータ
for x in np.linspace(-7, 7, 19):
    for y in np.linspace(-7, 7, 19):
        r = np.abs(np.abs(x) - np.abs(y))
        if r < 1.5:
            X4.append([x, y])
X4 = np.array(X4)

# サンプリングデータの図示
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 4))
for i, X in enumerate([X1, X2, X3, X4]):
    axes[i].scatter(X[:, 0], X[:, 1], label="X")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# ２次元関数をヒートマップにする関数
def heatmap(
    f,
    x_min=-10,
    x_max=10,
    y_min=-10,
    y_max=10,
    h=0.1,
    dtype=[0, 1, 1, 1],
    epsilon=None,
    drawline=False,
    cmap=plt.cm.jet,
    ax=None,
):
    def get_Z(f, dt):
        if dt == 1:
            X = []
            for y in np.arange(y_min, y_max, h):
                for x in np.arange(x_min, x_max, h):
                    X.append([x, y])
            X = np.array(X)
            Z = f(X).reshape(
                len(np.arange(x_min, x_max, h)), len(np.arange(y_min, y_max, h))
            )
        else:
            Z = [[f([x, y]) for x, y in zip(xx, yy)] for xx, yy in zip(x_mg, y_mg)]
        return Z

    x_mg, y_mg = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    if type(f) == list:
        for i, ff in enumerate(f):
            if dtype[i] == -1:
                if ax is not None:
                    pass  # ax.set_contour(x_mg, y_mg, get_Z(ff, 1), colors='black')
                else:
                    plt.contour(x_mg, y_mg, get_Z(ff, 1), colors="black")
            elif i == 0:
                Z = np.array(get_Z(ff, dtype[i]))
            else:
                Z -= np.array(get_Z(ff, dtype[i]))
        Z = Z.tolist()

    else:
        Z = get_Z(f, dtype[0])

    if epsilon:
        Z = np.array(Z)
        Z = np.ma.array(Z, mask=np.abs(Z) <= epsilon)
        cmap.set_bad((1, 1, 1, 1))  # 無効な値に対応する色
        # cmap.set_bad((0, 0, 0, 1))  # 無効な値に対応する色

    if ax is not None:
        # ax.set_aspect('equal')
        if drawline:
            ax.set_contour(x_mg, y_mg, Z, colors="black")
        ax.imshow(Z, origin="lower", extent=[x_min, x_max, y_min, y_max], cmap=cmap)
        # ax.set_colorbar()
        # plt.grid()
        # plt.show()
    else:
        plt.axes().set_aspect("equal")
        if drawline:
            plt.contour(x_mg, y_mg, Z, colors="black")
        plt.imshow(Z, origin="lower", extent=[x_min, x_max, y_min, y_max], cmap=cmap)
        plt.colorbar()
        # plt.grid()
        plt.show()

In [ ]:
# データ密度を計算するためのクラス

from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
from sklearn.svm import OneClassSVM


class KNN:
    def __init__(self, n_neighbors=5, outlier_fraction=0.1, algorithm="ball_tree"):
        self.n_neighbors = n_neighbors
        self.outlier_fraction = outlier_fraction
        self.model = False
        self.algorithm = algorithm
        self.distances = False
        self.indices = False
        self.threshold = False
        self.len_data = False

    def fit(self, X):
        self.len_data = len(X)
        self.model = NearestNeighbors(
            n_neighbors=self.n_neighbors, algorithm=self.algorithm
        ).fit(X)
        self.distances, self.indices = self.model.kneighbors(X)
        self.threshold = sorted(self.distances[:, self.n_neighbors - 1])[
            int((self.len_data - 1) * (1 - self.outlier_fraction))
        ]

    def transform(self, x):
        self.distances, self.indices = self.model.kneighbors(x)
        return self.distances[:, self.n_neighbors - 1]

    def transform_bin(self, x):
        self.transform(x)
        self.Z = self.distances[:, self.n_neighbors - 1]
        return np.where(self.Z >= self.threshold, 0, 1)


class AVDetector:
    def __init__(
        self,
        model=OneClassSVM(),
        outlier_fraction=0.1,
    ):
        self.model = model
        self.outlier_fraction = outlier_fraction
        self.threshold = False
        self.len_data = False

    def fit(self, X):
        self.len_data = len(X)
        self.model.fit(X)
        self.threshold = sorted(self.model.decision_function(X))[
            int((self.len_data - 1) * self.outlier_fraction)
        ]

    def transform(self, x):
        return self.model.decision_function(x)

    def transform_bin(self, x):
        self.Z = self.model.decision_function(x)
        return np.where(self.Z >= self.threshold, 1, 0)

In [ ]:
# ４種類のサンプリングデータに５種類のデータ密度計算法を適用して図示

avd_dict = {}
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(16, 20))
for i, X in enumerate([X1, X2, X3, X4]):
    for j, (avd_name, avd) in enumerate(
        [
            ["KNN", KNN()],
            ["OneClassSVM", AVDetector(model=OneClassSVM())],
            ["IsolationForest", AVDetector(model=IsolationForest())],
            ["LocalOutlierFactor", AVDetector(model=LocalOutlierFactor(novelty=True))],
            ["EllipticEnvelope", AVDetector(model=EllipticEnvelope())],
        ]
    ):
        avd.fit(X)
        avd_dict[avd_name] = avd
        axes[j][i].set_title(avd_name)
        axes[j][i].scatter(X[:, 0], X[:, 1], alpha=0.2)
        heatmap([avd.transform, avd.transform_bin], dtype=[1, -1], ax=axes[j][i])

## 真の関数

In [ ]:
# 機械学習で予測したい「真の関数」を４種類

f1 = (
    lambda x: (
        np.sqrt(
            np.abs(
                np.sqrt(
                    np.abs(
                        np.minimum(
                            (x[0] / 3) ** 2 + (x[1] / 3) ** 2 - 1,
                            (np.abs(x[0] / 3) - 0.95) ** 2
                            + (x[1] / 3 - 0.95) ** 2
                            - 0.55 ** 2,
                        )
                    )
                )
            )
        )
        + 1.2 / (1 + np.abs(x[0] ** 2 + (0.8 + x[1]) ** 2))
        - 0.8 / (1 + np.abs((1.1 + x[0]) ** 2 + (0.9 - 0.9 * x[1]) ** 2))
        - 0.8 / (1 + np.abs((1.1 - x[0]) ** 2 + (0.9 - 0.9 * x[1]) ** 2))
        - 1
    )
    / 0.37
)
f2 = (
    lambda x: (np.sqrt(np.abs(x[0] ** 2 + (x[1] - np.sqrt(np.abs(x[0]))) ** 2)) - 5.8)
    / 2.4
)
f3 = (
    lambda x: (np.linalg.norm([x[0], x[1]]) - np.cos(6 * np.arctan2(x[0], x[1])) - 114)
    / 0.7
)
f4 = lambda x: (np.linalg.norm([x[0], x[1]]) - 2 * np.arctan2(x[0], x[1]) - 114) / 3.6

In [ ]:
# 「真の関数」を図示
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 4))
axes[0].set_title("f1")
heatmap(f1, ax=axes[0])
axes[1].set_title("f2")
heatmap(f2, ax=axes[1])
axes[2].set_title("f3")
heatmap(f3, ax=axes[2])
axes[3].set_title("f4")
heatmap(f4, ax=axes[3])

## デフォルトパラメーターでフィッティング

In [ ]:
# 主要な９種類の機械学習手法で、４種類のサンプリングデータから、４種類の「真の関数」にフィッティング

from sklearn.ensemble import (
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

models = [
    ["LR", LinearRegression()],
    ["SVM", SVR()],
    ["KN", KNeighborsRegressor()],
    ["GP", GaussianProcessRegressor()],
    ["DT", DecisionTreeRegressor()],
    ["RF", RandomForestRegressor()],
    ["ET", ExtraTreesRegressor()],
    ["GB", HistGradientBoostingRegressor()],
    ["MLP", MLPRegressor()],
]

obtained_models = {}
for data_name, X in [["dense", X1], ["sparse", X2], ["O-shape", X3], ["X-shape", X4]]:
    fig, axes = plt.subplots(
        nrows=len(models) + 1, ncols=4, figsize=(16, 4 * len(models) + 1)
    )
    for i, f in enumerate([f1, f2, f3, f4]):
        axes[0][i].set_title("{} f_{}".format(data_name, i + 1))
        axes[0][i].scatter(X[:, 0], X[:, 1], alpha=0.1)
        axes[0][i].xaxis.set_visible(False)
        axes[0][i].yaxis.set_visible(False)
        heatmap(f, ax=axes[0][i])
        for j, (model_name, model) in enumerate(models):
            model.fit(X, f(X.T))
            axes[j + 1][i].set_title("{} f_{}, {}".format(data_name, i + 1, model_name))
            axes[j + 1][i].xaxis.set_visible(False)
            axes[j + 1][i].yaxis.set_visible(False)
            heatmap(model.predict, dtype=[1], ax=axes[j + 1][i])
            obtained_models[(data_name, i, j)] = model
    plt.show()

## Optuna によるハイパーパラメーターチューニング

In [ ]:
from functools import wraps


# 学習に時間がかかりすぎる場合に強制終了するための方法
def on_timeout(limit, handler, hint=None):
    def notify_handler(signum, frame):
        handler(
            "'%s' terminated since it did not finish in %d seconds." % (hint, limit)
        )

    def __decorator(function):
        def __wrapper(*args, **kwargs):
            import signal

            signal.signal(signal.SIGALRM, notify_handler)
            signal.alarm(limit)
            result = function(*args, **kwargs)
            signal.alarm(0)
            return result

        return wraps(function)(__wrapper)

    return __decorator


def handler_func(msg):
    print(msg)

In [ ]:
import copy

from sklearn.metrics import mean_absolute_error


# Optunaでチューニングするための基本クラス
class BestTune:
    def __init__(self, x_train, x_valid, t_train, t_valid):
        # 訓練データを格納
        self.x_train = x_train
        self.t_train = t_train

        # 検証データを格納
        self.x_valid = x_valid
        self.t_valid = t_valid

        # ベストモデルとスコアを格納
        self.best_score = None
        self.best_model = None

    def get_model_params(self, trial):
        raise NotImplementedError()

    def get_base_model(self):
        raise NotImplementedError()

    @on_timeout(limit=learning_time_limit, handler=handler_func, hint=u"BestTune")
    def __call__(self, trial):
        model_params = self.get_model_params(trial)
        model = self.get_base_model()(**model_params)
        model.fit(self.x_train, self.t_train)

        # 検証データの予測性能を評価
        score = mean_absolute_error(model.predict(self.x_valid), self.t_valid)

        # ベストスコアが出れば、そのベストモデルを記録
        if self.best_model is None or self.best_score > score:
            self.best_score = score
            self.best_model = copy.deepcopy(model)

        # スコアを返す
        return score

In [ ]:
# Support Vector Machine Regressor
class best_SVR(BestTune):
    def get_base_model(self):
        return SVR

    def get_model_params(self, trial):
        # チューニングしたいパラメータの範囲を設定
        model_params = {}
        model_params["C"] = trial.suggest_float("C", 1e-10, 1e10, log=True)
        model_params["gamma"] = trial.suggest_float("gamma", 1e-10, 1e10, log=True)
        model_params["epsilon"] = trial.suggest_float("epsilon", 1e-10, 1e10, log=True)
        model_params["max_iter"] = 530000
        return model_params

In [ ]:
# K-Neighbors Regressor
class best_KN(BestTune):
    def get_base_model(self):
        return KNeighborsRegressor

    def get_model_params(self, trial):
        model_params = {}
        model_params["algorithm"] = trial.suggest_categorical(
            "algorithm", ["ball_tree", "kd_tree", "brute"]
        )
        model_params["n_neighbors"] = trial.suggest_int("n_neighbors", 1, 10)
        model_params["weights"] = trial.suggest_categorical(
            "weights", ["uniform", "distance"]
        )
        return model_params

In [ ]:
# Decision Tree Regressor
class best_DT(BestTune):
    def get_base_model(self):
        return DecisionTreeRegressor

    def get_model_params(self, trial):
        model_params = {}
        model_params["max_depth"] = trial.suggest_int("max_depth", 1, 10)
        model_params["min_samples_leaf"] = trial.suggest_int("min_samples_leaf", 1, 10)
        model_params["criterion"] = trial.suggest_categorical(
            "criterion", ["squared_error", "friedman_mse", "absolute_error"]
        )
        return model_params

In [ ]:
# Random Forest Regressor
class best_RF(BestTune):
    def get_base_model(self):
        return RandomForestRegressor

    def get_model_params(self, trial):
        model_params = {}
        model_params["max_depth"] = trial.suggest_int("max_depth", 1, 10)
        model_params["min_samples_leaf"] = trial.suggest_int("min_samples_leaf", 1, 10)
        model_params["criterion"] = trial.suggest_categorical(
            "criterion", ["squared_error", "friedman_mse", "absolute_error"]
        )
        return model_params

In [ ]:
# ExtraTrees Regressor
class best_ET(BestTune):
    def get_base_model(self):
        return ExtraTreesRegressor

    def get_model_params(self, trial):
        model_params = {}
        model_params["max_depth"] = trial.suggest_int("max_depth", 1, 10)
        model_params["min_samples_leaf"] = trial.suggest_int("min_samples_leaf", 1, 10)
        model_params["criterion"] = trial.suggest_categorical(
            "criterion", ["squared_error", "friedman_mse", "absolute_error"]
        )
        return model_params

In [ ]:
# GradientBoosting Regressor
class best_GB(BestTune):
    def get_base_model(self):
        return HistGradientBoostingRegressor

    def get_model_params(self, trial):
        model_params = {}
        model_params["max_depth"] = trial.suggest_int("max_depth", 1, 10)
        model_params["min_samples_leaf"] = trial.suggest_int("min_samples_leaf", 1, 10)
        model_params["loss"] = trial.suggest_categorical(
            "loss", ["squared_error", "absolute_error"]
        )
        return model_params

In [ ]:
# Multi-Layer Perceptron Regressor
class best_MLP(BestTune):
    def get_base_model(self):
        return MLPRegressor

    def get_model_params(self, trial):
        model_params = {}
        n_layer = trial.suggest_int("n_layer", 1, 10)
        in_neuron = trial.suggest_int("in_neuron", 1, 200)
        mid_neuron = trial.suggest_int("mid_neuron", 1, 200)
        out_neuron = trial.suggest_int("out_neuron", 1, 200)
        model_params["hidden_layer_sizes"] = (
            [in_neuron] + [mid_neuron] * n_layer + [out_neuron]
        )
        model_params["activation"] = trial.suggest_categorical(
            "activation", ["logistic", "tanh", "relu"]
        )
        model_params["learning_rate"] = trial.suggest_categorical(
            "learning_rate", ["constant", "invscaling", "adaptive"]
        )
        # model_params["solver"] = trial.suggest_categorical(
        #    "solver", ["lbfgs", "adam"]
        # )
        model_params["early_stopping"] = trial.suggest_categorical(
            "early_stopping", [True, False]
        )
        model_params["max_iter"] = 530000
        return model_params

In [ ]:
# 偶数番目のデータを教師データ、奇数番目のデータを検証データとする


def train_test_split(X):
    index = np.arange(X.shape[0])
    train_index = index[index % 2 == 0]
    test_index = index[index % 2 != 0]
    X_train = X[train_index, :]
    X_test = X[test_index, :]
    return X_train, X_test

In [ ]:
import optuna


# Optuna で学習を繰り返し、学習履歴を保存する
def train(
    study_name,
    f,
    tune_model,
    X_train,
    X_test,
    timeout=timeout_optuna,
    n_trials=n_trials_optuna,
    show_progress_bar=True,
):
    import warnings

    warnings.simplefilter("ignore")
    optuna.logging.set_verbosity(optuna.logging.WARN)
    best_model = tune_model(X_train, X_test, f(X_train.T), f(X_test.T))

    # 学習環境を立ち上げる
    study = optuna.create_study(
        study_name=study_name,
        storage="sqlite:///" + study_name + ".sql",
        load_if_exists=True,
        direction="minimize",
    )
    try:
        study.enqueue_trial(study.best_trial.params)
    except:
        pass

    # 学習する
    study.optimize(
        best_model,
        timeout=timeout,
        n_trials=n_trials,
        show_progress_bar=show_progress_bar,
    )
    return best_model

In [ ]:
# ７種類の機械学習手法で、４種類のサンプリングデータから、４種類の「真の関数」を学習
for data_name, X in [
    ["O-shape", X3],
    ["X-shape", X4],
    ["dense", X1],
    ["sparse", X2],
]:
    for func_name, f in [["f1", f1], ["f2", f2], ["f3", f3], ["f4", f4]]:
        for model_name, tune_model in [
            ["SVM", best_SVR],
            ["KN", best_KN],
            ["DT", best_DT],
            ["RF", best_RF],
            ["ET", best_ET],
            ["GB", best_GB],
            ["MLP", best_MLP],
        ]:
            strage_name = "_".join([model_name, func_name, data_name, dateflag])
            X_train, X_test = train_test_split(X)
            overfit = train(
                "{}{}_overfit".format(MODEL_PATH, strage_name), f, tune_model, X, X
            )
            bestfit = train(
                "{}{}_bestfit".format(MODEL_PATH, strage_name),
                f,
                tune_model,
                X_train,
                X_test,
            )

            fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 8))
            axes[0][0].set_title(func_name)
            axes[1][0].set_title(func_name)
            axes[0][1].set_title(
                "_".join([model_name, func_name, data_name, "overfit"])
            )
            axes[0][2].set_title(
                "_".join([model_name, func_name, data_name, "overfit", "diff"])
            )
            axes[1][1].set_title(
                "_".join([model_name, func_name, data_name, "bestfit"])
            )
            axes[1][2].set_title(
                "_".join([model_name, func_name, data_name, "bestfit", "diff"])
            )
            heatmap(f, ax=axes[0][0])
            axes[0][0].scatter(X[:, 0], X[:, 1], alpha=0.1)
            heatmap(f, ax=axes[1][0])
            axes[1][0].scatter(X_train[:, 0], X_train[:, 1], alpha=0.1)
            heatmap(overfit.best_model.predict, dtype=[1], ax=axes[0][1])
            heatmap(
                [f, overfit.best_model.predict],
                dtype=[0, 1],
                epsilon=0.1,
                ax=axes[0][2],
            )
            heatmap(bestfit.best_model.predict, dtype=[1], ax=axes[1][1])
            heatmap(
                [f, bestfit.best_model.predict],
                dtype=[0, 1],
                epsilon=0.1,
                ax=axes[1][2],
            )
            axes[0][3].set_title(
                "MAE={:.4f}".format(
                    mean_absolute_error(f(X.T), overfit.best_model.predict(X))
                )
            )
            axes[0][3].scatter(f(X.T), overfit.best_model.predict(X), alpha=0.5)
            axes[1][3].set_title(
                "MAE(train, test)={:.4f}, {:.4f}".format(
                    mean_absolute_error(
                        f(X_train.T), bestfit.best_model.predict(X_train)
                    ),
                    mean_absolute_error(
                        f(X_test.T), bestfit.best_model.predict(X_test)
                    ),
                )
            )
            axes[1][3].scatter(
                f(X_train.T),
                bestfit.best_model.predict(X_train),
                alpha=0.5,
                label="train",
            )
            axes[1][3].scatter(
                f(X_test.T), bestfit.best_model.predict(X_test), alpha=0.5, label="test"
            )
            axes[1][3].legend()
            plt.savefig(
                "{}{}.png".format(MODEL_PATH, strage_name), format="png", dpi=300
            )
            plt.show()
            print("Overfit model=", overfit.best_model)
            print("Bestfit model=", bestfit.best_model)

## ハイパーパラメーターの解析

In [ ]:
# Optunaによる学習履歴を記録することで、ハイパーパラメーターの影響を可視化できます
import glob

import matplotlib.pyplot as plt
import numpy as np
import optuna

for file in sorted(glob.glob(MODEL_PATH + "*fit.sql")):
    print(file)
    study = optuna.create_study(
        study_name=file.replace(".sql", ""),
        storage="sqlite:///" + file,
        load_if_exists=True,
        direction="minimize",
    )
    importances = np.array(
        [[k, v] for k, v in optuna.importance.get_param_importances(study).items()]
    )
    optuna.visualization.plot_param_importances(study).show()
    optuna.visualization.plot_contour(
        study, params=[importances[0, 0], importances[1, 0]]
    ).show()
    # break